In [21]:
from utils.simulation.sim_utils import RandomSimulator
from utils.core.save_manager import SaveUtils
from utils.data_io import load_data
import os

🧪 Simulating Unscaled Normally Distributed Data with Specified Skewness and Kurtosis

In [22]:
simulator = RandomSimulator(num_simulations=1000)
df = simulator.simulate_normal(num_variables=10, target_skew=0, target_kurt=3)

save_util = SaveUtils()
save_util.save_dataframe_to_csv(df,os.path.join(os.getcwd(), "data/output/simulated_normal.csv"), overwrite=True)

Data saved to c:\Data Science Projects\Daanish\projects\simulation\data/output/simulated_normal.csv successfully.


🧪 Simulate scaled normals (e.g., volatilities)

In [23]:
# params = [0.2, 0.15, 0.3]  # std devs or volatilities
# simulator = RandomSimulator(parameters=params)
# df = simulator.simulate_normal(target_skew=0, target_kurt=3)

📊 Simulating Poisson Data from Excel Input
- Reads input data from an Excel file
- Initializes a Poisson simulator
- Simulates Poisson-distributed values
- Saves the simulated data to an Excel file

In [24]:
df = load_data(
    source_type='excel',
    input_path= os.path.join(os.getcwd(), "data/input/Poisson Simulation.xlsx"),
    sheet_name='Lambda Calculation',
    usecols=['Lambda']
)
simulator = RandomSimulator(parameters=df['Lambda'],num_simulations = 10000)
sim = simulator.simulate_poisson()
save_util.save_dataframe_to_excel(
    sim,
    os.path.join(os.getcwd(), "data/output/simulated_poisson.xlsx"),
    sheet_name="simulated values", 
    overwrite=True
    )


Excel file saved to c:\Data Science Projects\Daanish\projects\simulation\data/output/simulated_poisson.xlsx successfully.
